In [1]:
import openml
from sklearn import preprocessing
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt

import torch
import sklearn

from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import datasets, transforms
from ucimlrepo import fetch_ucirepo 

import utils.losses as losses

In [5]:

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

import utils.losses as losses
# Load Iris dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Labels

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_train_tensor = torch.eye(3)[y_train_tensor] 
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define Logistic Regression model for Multiclass Classification
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)  # Softmax will be applied by the loss function

# Initialize model, loss function, and optimizer
input_dim = X_train.shape[1]
output_dim = len(iris.target_names)  # Number of classes
model = LogisticRegressionModel(input_dim=input_dim, output_dim=output_dim)
loss_fn = losses.FwdLoss(torch.eye(3))
#criterion = nn.CrossEntropyLoss()  # Cross-Entropy Loss for multiclass classification
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        # Forward pass
        y_batch = torch.max(y_batch,dim=1).indices
        outputs = model(X_batch).to(device)
        print(outputs)
        print(y_batch)
        loss = loss_fn(outputs, y_batch.to(device))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():
    y_pred_train = model(X_train_tensor)
    y_pred_test = model(X_test_tensor)

    # Convert logits to class labels
    _, y_pred_train = torch.max(y_pred_train, 1)
    _, y_pred_test = torch.max(y_pred_test, 1)

    # Calculate accuracy
    train_accuracy = (y_pred_train == y_train_tensor).float().mean()
    test_accuracy = (y_pred_test == y_test_tensor).float().mean()

    print(f'Train Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')


tensor([[-0.3864, -0.6689,  0.5652],
        [ 0.3272, -0.5891,  0.4798],
        [ 0.1113, -0.6197,  0.4614],
        [ 1.5977,  0.8957,  0.4778],
        [ 0.3292, -0.8456,  0.4270],
        [-0.1138, -1.0523,  0.4472],
        [ 1.5552,  1.0163,  0.5072],
        [ 1.5086,  0.4223,  0.4482],
        [ 0.2670, -0.8089,  0.4505],
        [-0.1777, -1.2932,  0.4346],
        [ 0.2839, -0.4365,  0.4980],
        [ 0.4973, -0.2272,  0.5107],
        [ 0.1184, -0.5768,  0.4631],
        [ 0.2038, -0.5072,  0.4994],
        [-0.4572, -0.9123,  0.5122],
        [-0.4295, -0.8585,  0.5347]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)
tensor([2, 1, 1, 0, 1, 1, 0, 0, 1, 2, 1, 1, 1, 1, 2, 2])


c:\Users\danib\VSprojects\ForwardBackard_losses\utils\losses.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.M = torch.tensor(M, dtype=torch.float32, device=device)


UnboundLocalError: cannot access local variable 'targets' where it is not associated with a value

In [ ]:
import numpy as np
p = (torch.tensor([1,1,1])/3).to(device)
ou = outputs@p.T

C:\Users\danib\AppData\Local\Temp\ipykernel_39508\4175460708.py:3: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3701.)
  ou = outputs@p.T


In [ ]:
ou

tensor([-0.5651,  0.0473, -0.4612, -0.0514,  1.1397,  0.1873, -0.6987,  0.5616,
         0.8801, -0.5733,  0.7952, -0.1659,  0.9018,  0.7871,  0.6841, -0.3042],
       device='cuda:0', grad_fn=<MvBackward0>)

In [ ]:
outputs[y_batch,range(outputs.size(1))]

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [16], [3]

In [ ]:
y_batch = torch.max(y_batch,dim=1).indices

In [ ]:
y_batch

tensor([2, 1, 2, 1, 0, 1, 2, 0, 0, 2, 0, 1, 0, 0, 0, 2])

In [ ]:
softmax = torch.nn.Softmax(dim=1)